# Notebook #1: Extracción y Transformación

- En este notebook realizaremos extraeremos datos haciendo uso de webscraping y APIs, para las sigueintes fuentes de información:
1. API AirBnB: datos de viviendas en régimen de alquiler turístico.
2. Webscraping de Redpiso: datos de viviendas en alquiler.
3. API Idealista: datos de viviendas en alquiler.
4. Webscraping del INE: datos sobre ingresos por hogar.
5. Webscraping del Ayuntamiento de Madrid: datos sobre población y cantidad de extranjeros.

- Usando este mismo notebook, se realizará la limpieza/transformación de los datos que cargaremos posteriormente en nuestra base de datos. Todas las funciones aquí utilizadas encuentran su soporte en `../src/soporte_funciones.py`

- Dado que los datos se modifican cada vez que se ejecuta una función de consulta, las mismas están comentadas. Para ejecutarlas, debe eliminarse la #, con la consecuencia de que los datos de origen serán sustituidos.

- El primer paso será importar las librerías necesarias:

In [2]:
# Librerías para tratamiento de datos

import pandas as pd
import geopandas as gpd
pd.set_option('display.max_columns', None) # Parámetro que modifica la visualización de los DFs
import numpy as np
import re

# Librería para el acceso a variables y funciones
import sys
sys.path.append("../")
from src import soporte_funciones as sf #Archivo .py donde encontraremos todas nuestras funciones
#from src import soporte_variables as sv

# Librería para acceder a funcionalidades del sistema operativo
import os

# Librerías para trabajar con distintos formatos de archivos
import pickle
import json

# Librería para ignorar avisos
import warnings
warnings.filterwarnings("ignore") # Ignora TODOS los avisos

/Users/davidfranco/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## API AirBnB

- Obtendremos datos de alquileres turísticos a través de una API de AirBnB.
- De la consulta, que realizamos con la función `sf.consulta_airbnbs()`, obtendremos una lista correspondiente a cada página de respuesta.
- La función recibe como argumentos la ciudad de destino, las fechas de entrada y salida (formato año-mes-día: xxxx-xx-xx) y el número de páginas de resultados que deseamos consultar (40 resultados por página).
- Dadas las limitaciones de la función cada iteración tarda unos 22 segundos.

In [ ]:
#resultados_airbnb = sf.consulta_airbnbs("Madrid", "2025-01-31", "2025-02-02", 11)

100%|██████████| 11/11 [03:56<00:00, 21.52s/it]


- La consulta devuelve el resultado para las fechas elegidas (31 de enero al 2 de febrero de 2025 para 2 adultos). Comprobamos que se hayan recibido 11 resultados y almacenamos la lista en un archivo json.

In [ ]:
#len(resultados_airbnb)

11

In [ ]:
#with open("../datos/origen/airbnb.json", "w") as json_file:
#    json.dump(resultados_airbnb, json_file, indent=4)

- Ahora importaremos el resultado de la consulta y le aplicaremos la función `dataframe_airbnb()`, que recibe como argumento el archivo json con los resultados y devuelve un dataframe con los datos tranformados.

In [4]:
with open("../datos/origen/airbnb.json", 'r') as file:
    resultados_airbnb = json.load(file)

In [9]:
df_airbnb = sf.dataframe_airbnb(resultados_airbnb)

- La función devuelve un dataframe con los resultados transformados, comprobamos que tiene 359 alojamientos en 4 columnas.

In [10]:
df_airbnb.shape

(360, 4)

- La respuesta contiene descripciones en inglés. Para traducirlas, usaremos la función `traducir_es()`, que recibe un texto y lo traduce a español haciendo uso de Google Translate. La aplicaremos a la columna "Descripción", e imprimos una muestra del resultado.

In [11]:
df_airbnb["Descripcion"] = df_airbnb["Descripcion"].apply(sf.traducir_es)

In [12]:
df_airbnb.sample(3)

,Latitud,Longitud,Descripcion,Precio Total
123,40.422270,-3.701760,Habitación grande privada con 3 balcones.Gran vía,204
232,40.429271,-3.695078,Loft de lujo en la calle Almagro,339
308,40.420123,-3.698174,Madrid flats Chueca,397


- Ahora vamos a convertir la latitud y la longitud a geopuntos. Cambiamos también el formato de CRS (Coordinate Reference System), para estandarizar el formato con los datos obtenidos de los municipios de Madrid, y guardamos en un archivo de tipo `geojason`.

In [ ]:
gdf_airbnb = gpd.GeoDataFrame(df_airbnb, geometry=gpd.points_from_xy(df_airbnb.Longitud, df_airbnb.Latitud))
gdf_airbnb.crs = "EPSG:4326"
#gdf_airbnb.to_file('../datos/origen/airbnb.geojson', driver='GeoJSON')

- Para poder asignar las coordenadas de cada vivienda o alojamiento a un distrito, será necesaria la información geográfica sobre cada uno de ellos.
- Tras una búsqueda en Google, hemos conseguido descargar los polígonos que representan cada distrito de Madrid. Cargamos la información en un DF.

In [14]:
gdf_distritos = gpd.read_file("../datos/origen/madrid-districts.geojson")

- Realizamos algunas tareas de limpieza:

In [15]:
gdf_distritos = gdf_distritos.rename(columns={"name":"Distrito","cartodb_id":"ID_Distrito"})
gdf_distritos.drop(columns= ["created_at", "updated_at"], inplace=True)

- Y lo unimos con nuestro DF de AirBnB, de modo que, ahora cada punto geográfico está contenido dentro de un municipio.

In [16]:
gdf_sjoin = gpd.sjoin(gdf_airbnb, gdf_distritos, how="inner", predicate="within")
gdf_sjoin = gdf_sjoin.drop(columns="index_right")

In [17]:
gdf_sjoin.head(1)

,Latitud,Longitud,Descripcion,Precio Total,geometry,Distrito,ID_Distrito
0,40.411549,-3.697992,Acogedor baño de doble baño en el corazón de M...,186,POINT (-3.69799 40.41155),Centro,1


- Reordenamos las columnas, comprobamos los tipos de datos y guardamos el resultado final en un archivo de tipo `geojson`, que cargaremos posteriormente a la base de datos de Postgres.

In [47]:
gdf_sjoin_final = gdf_sjoin[["ID_Distrito", "Precio Total", "Descripcion", "geometry"]]

In [48]:
gdf_sjoin_final.sample(2)

,ID_Distrito,Precio Total,Descripcion,geometry
54,15,82,Habitación acogedora a los 15-20 min.desde el ...,POINT (-3.64209 40.43146)
243,1,385,Aparte.Sol central,POINT (-3.70231 40.41898)


In [169]:
gdf_sjoin_final.dtypes

ID_Distrito        int32
Precio Total       int64
Descripcion       object
geometry        geometry
dtype: object

In [ ]:
#gdf_sjoin_final.to_file('../datos/finales/airbnb.geojson', driver='GeoJSON')
#gdf_sjoin.to_file('../datos/finales/airbnb.shp')

## Creación del DF Distritos
- En él se listan los distritos de Madrid junto con los códigos asignados por el ayuntamiento, así como los polígonos, que contienen los límites geográficos de cada uno.
- Reordenamos las columnas, comprobamos tipos de datos y lo guardamos nuevamente en un archivo `geojson`, que nos permitirá mantener el tipo de dato geográfico.

In [33]:
df_distritos = gdf_distritos[["ID_Distrito", "Distrito", "geometry"]]
#df_distritos.to_file('../datos/finales/distritos.geojson', driver='GeoJSON')
df_distritos

,ID_Distrito,Distrito,geometry
0,1,Centro,"MULTIPOLYGON (((-3.69185 40.40853, -3.69189 40..."
1,2,Arganzuela,"MULTIPOLYGON (((-3.70258 40.40638, -3.70166 40..."
2,3,Retiro,"MULTIPOLYGON (((-3.66279 40.4097, -3.66384 40...."
3,4,Salamanca,"MULTIPOLYGON (((-3.65809 40.43945, -3.65828 40..."
4,5,Chamartin,"MULTIPOLYGON (((-3.67231 40.48388, -3.67237 40..."
5,6,Tetuan,"MULTIPOLYGON (((-3.69633 40.47572, -3.69619 40..."
6,7,Chamberi,"MULTIPOLYGON (((-3.68991 40.44737, -3.69048 40..."
7,8,Fuencarral-El Pardo,"MULTIPOLYGON (((-3.64131 40.63922, -3.64118 40..."
8,9,Moncloa-Aravaca,"MULTIPOLYGON (((-3.79973 40.47063, -3.79887 40..."
9,10,Latina,"MULTIPOLYGON (((-3.7213 40.41256, -3.72051 40...."


In [168]:
df_distritos.dtypes

ID_Distrito       int32
Distrito         object
geometry       geometry
dtype: object

## WebScraping de Redpiso

- Obtendremos datos de alquileres de viviendas haciendo uso de Selenium y BeautifulSoup, para navegar y obtener el código html de la web de Redpiso, que contiene información sobre pisos de alquiler en Madrid.
- De la consulta, que realizamos con la función `sf.consulta_alquileres_redpiso()`, obtendremos una lista de "sopas" de BeautifulSoup, correspondiente a cada página.
- La función recibe como argumento el número de páginas de resultados que queremos guardar. En este caso, serán 50. La lista de sopas la almacenaremos en un archivo de tipo Pickle.

In [ ]:
#sopas_redpiso = sf.scraping_alquileres_redpiso(50)

In [ ]:
#with open('../datos/origen/sopas_redpiso.pkl', 'wb') as file:
#    pickle.dump(sopas_redpiso, file)

- Ahora importaremos el resultado de la consulta y le aplicaremos la función `dataframe_redpiso()`, que recibe como argumento el archivo pickle con las sopas.

In [21]:
with open('../datos/origen/sopas_redpiso.pkl', 'rb') as file:
    sopas_redpiso = pickle.load(file)

In [22]:
df_redpiso = sf.dataframe_redpiso(sopas_redpiso)

- Vemos que, hemos obtenido un DF con un total de 599 viviendas en alquiler.

In [23]:
df_redpiso.shape

(600, 2)

- Procederemos ahora a limpiar los datos:
1. Aplicaremos la función `extraer_distrito()` a la columna descripción. Esta función aplica un patrón de Regex al string para obtener únicamente el distrito. Si no lo encuentra, devuelve "Distrito no identificado". Limpiaremos esos registros no identificados posteriormente.
2. En la columna precio sustituimos los puntos, los signos de euro y los strings "a consultar".
3. Homogenizamos los nombres de los distritos eliminando las tildes y los nombres generales. 

In [24]:
df_redpiso["Distrito"] = df_redpiso['Descripcion'].apply(sf.extraer_distrito)
df_redpiso["Descripcion"] = df_redpiso["Descripcion"].str.title()
df_redpiso["Precio"] = df_redpiso["Precio"].str.replace("A consultar","0")
df_redpiso["Precio"] = df_redpiso["Precio"].str.replace(".","")
df_redpiso["Precio"] = df_redpiso["Precio"].str.replace(" €","").astype(int)
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("Villa de Vallecas-Ensanche y Santa Eugenia","Puente de Vallecas")
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("San Blas-Canillejas","San Blas")
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("Vicálvaro-Ambroz-Centro-Valdebernardo-Valderribas","Vicálvaro")
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("Chamberí","Chamberi")
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("Chamartín","Chamartin")
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("Vicálvaro","Vicalvaro")
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("Tetuán","Tetuan")

- Tras aplicar el patrón de Regex, nos hemos quedado con 20 entradas sin identificar. Asignaremos esos distritos manualmente y comprobaremos que se hayan eliminado.

In [25]:
df_redpiso["Distrito"][0] = "Villaverde"
df_redpiso["Distrito"][13] = "Moncloa-Aravaca"
df_redpiso["Distrito"][39] = "Moncloa-Aravaca"
df_redpiso["Distrito"][112] = "Moncloa-Aravaca"
df_redpiso["Distrito"][187] = "Villaverde"
df_redpiso["Distrito"][235] = "Centro"
df_redpiso["Distrito"][257] = "Puente de Vallecas"
df_redpiso["Distrito"][273] = "Centro"
df_redpiso["Distrito"][288] = "Moncloa-Aravaca"
df_redpiso["Distrito"][380] = "Usera"
df_redpiso["Distrito"][387] = "Moncloa-Aravaca"
df_redpiso["Distrito"][446] = "Centro"
df_redpiso["Distrito"][464] = "Moncloa-Aravaca"
df_redpiso["Distrito"][493] = "Centro"
df_redpiso["Distrito"][501] = "Centro"
df_redpiso["Distrito"][527] = "Chamberi"
df_redpiso["Distrito"][551] = "Chamartin"
df_redpiso["Distrito"][568] = "Moncloa-Aravaca"
df_redpiso["Distrito"][591] = "Moncloa-Aravaca"
df_redpiso["Distrito"][592] = "Moncloa-Aravaca"

In [26]:
df_redpiso[df_redpiso["Distrito"].str.contains("Distrito no identificado", case=False, na=False)]

,Descripcion,Precio,Distrito


- Comprobamos nuevamente que los nombres de los distritos son consistentes y no tenemos registros inesperados.

In [27]:
df_redpiso["Distrito"].unique()

array(['Villaverde', 'Chamberi', 'Salamanca', 'Ciudad Lineal',
       'Puente de Vallecas', 'Barajas', 'San Blas', 'Arganzuela',
       'Tetuan', 'Moncloa-Aravaca', 'Latina', 'Hortaleza',
       'Fuencarral-El Pardo', 'Retiro', 'Centro', 'Carabanchel',
       'Chamartin', 'Moratalaz', 'Usera', 'Vicalvaro'], dtype=object)

- Eliminamos los registros donde el precio del alquiler sea cero, debido a que distorsionarían el análisis.

In [28]:
df_redpiso["Precio"].value_counts()

Precio
1100    53
1200    51
0       43
850     42
1000    40
        ..
2350     1
550      1
690      1
720      1
1480     1
Name: count, Length: 83, dtype: int64

In [29]:
df_redpiso = df_redpiso[
    (df_redpiso["Precio"] != 0)
]

- Tras eliminar los registros, nos quedamos con 556 pisos repartidos en 20 distritos, de un total de 21 que tenemos en Madrid.

In [30]:
df_redpiso["Distrito"].value_counts()

Distrito
Chamberi               74
Chamartin              69
Centro                 62
Fuencarral-El Pardo    47
Hortaleza              44
Puente de Vallecas     43
Arganzuela             35
Salamanca              33
Latina                 31
Moncloa-Aravaca        26
Tetuan                 25
Retiro                 17
Carabanchel            15
San Blas               11
Villaverde              8
Ciudad Lineal           6
Vicalvaro               6
Barajas                 2
Moratalaz               2
Usera                   1
Name: count, dtype: int64

- Para asignar el ID a cada municipio, realiaremos un merge con el dataframe de distritos. 

In [34]:
df_redpiso_merge = df_redpiso.merge(df_distritos, how="inner", left_on="Distrito", right_on="Distrito")

- Eliminamos las columnas que no necesitamos y reorganizamos las restantes.

In [35]:
df_redpiso_merge.drop(columns = ["Distrito", "geometry"], inplace=True)
df_redpiso_merge = df_redpiso_merge[["ID_Distrito", "Precio", "Descripcion"]]
df_redpiso_merge.head(5)

,ID_Distrito,Precio,Descripcion
0,17,850,"Piso En Alquiler En Villaverde, Madrid, Madrid"
1,7,1100,"Piso En Alquiler En Calle Cristobal Bordiu, Rí..."
2,4,2000,"Apartamento En Alquiler En Calle Fundadores, F..."
3,15,744,"Piso En Alquiler En Calle Pepe Isbert, Pueblo ..."
4,13,750,Estudio En Alquiler En Calle Embalse De Navace...


- Comprobamos tipos de datos y guardamos el dataframe en un archivo de tipo CSV.

In [165]:
df_redpiso_merge.dtypes

ID_Distrito     int32
Precio          int64
Descripcion    object
dtype: object

In [ ]:
#df_redpiso_merge.to_csv("../datos/finales/redpiso.csv")

## API Idealista

- Obtendremos datos de viviendas en alquiler a través de una API de idealista.
- De la consulta, que realizamos con la función `sf.consulta_idealista()`, obtendremos una lista correspondiente a cada página de respuesta.
- La función recibe como argumentos el código, el nombre de la ciudad que deseamos consultar y el número de páginas que deseamos consultar, devolviendo 40 resultados por página. La lista de resultados la almacenaremos en un archivo de tipo `json`.
- Dadas las limitaciones de la función cada iteración tarda unos 20 segundos.

In [ ]:
#resultados_idealista = sf.consulta_idealista("0-EU-ES-28-07-001-079", "Madrid", 10)

In [ ]:
#with open("../datos/origen/idealista.json", "w") as json_file:
#    json.dump(resultados_idealista, json_file, indent=4)

- Ahora importaremos el resultado de la consulta, comprobaremos que la longitud de la lista sea correcta y le aplicaremos la función `dataframe_idealista()`, que recibe como argumento el archivo json con los resultados y devuelve un dataframe con los datos transformados.

In [12]:
with open("../datos/origen/idealista.json", 'r') as file:
    resultados_idealista = json.load(file)

In [15]:
len(resultados_idealista)

10

In [13]:
df_idealista = sf.dataframe_idealista(resultados_idealista)

- Eliminamos tildes en los nombres de las columnas y usamos nuevamente nuestra función de traducción sobre la columna "Tipo", obteniendo como resultado el DF que se imprime, que contiene información de 399 viviendas.

In [14]:
df_idealista["Tipo"] = df_idealista["Tipo"].apply(sf.traducir_es)
df_idealista["Direccion"] = df_idealista["Direccion"].str.title()
df_idealista["Descripcion"] = df_idealista["Descripcion"].str.title()

In [16]:
df_idealista.shape

(400, 10)

In [ ]:
df_idealista.head(2)

,Latitud,Longitud,Precio,Tipo,Planta,Tamaño,Habitaciones,Baños,Dirección,Descripción
0,40.432201,-3.714278,1350.0,departamento,1,73.0,2,1,"Calle De Andres Mellado, 18",Piso Con Una Habitación Con Cama De Matrimonio...
1,40.414669,-3.705414,2300.0,departamento,5,95.0,2,2,Calle Del Marqués Viudo De Pontejos,S C A L A | Properties Presenta Ático Amuebl...


- Cuando un valor se repite muchas veces, como puede ser el caso del tipo de vivienda, para crear nuestra base de datos, lo correcto sería obtener una nueva tabla con identificadores únicos para cada tipo de vivienda, así, normalizamos los datos, eliminando redundancias. Por una limitación de tiempo, no se hará en este proyecto.

In [52]:
df_idealista["Tipo"].value_counts()

Tipo
departamento    321
estudio          37
ático            25
dúplex           11
chalet            6
Name: count, dtype: int64

- Para poder asignar las coordenadas de cada vivienda a un distrito, transformamos la latitud y la longitud de entrada a un geopunto, y uniremos el nuevo GDF con nuestro GDF de distritos. Asignamos el tipo de CRS y guardamos en un archivo de tipo `geojson`.

In [ ]:
gdf_idealista = gpd.GeoDataFrame(df_idealista, geometry=gpd.points_from_xy(df_idealista.Longitud, df_idealista.Latitud))
gdf_idealista.crs = "EPSG:4326"
#gdf_idealista.to_file('../datos/origen/idealista.geojson', driver='GeoJSON')

In [41]:
gdf_sjoin2 = gpd.sjoin(gdf_idealista, gdf_distritos, how="inner", predicate="within")
gdf_sjoin2 = gdf_sjoin2.drop(columns="index_right")


- Realizamos algunas tareas más de limpieza, asignación de tipos de datos y guardamos el resultado final en un archivo `geojson`.

In [163]:
gdf_sjoin2_final = gdf_sjoin2[['ID_Distrito', 'Precio', 'Tipo', 'Planta', 'Tamanio',
       'Habitaciones', 'Banios', 'Direccion', 'Descripcion', 'geometry']]
gdf_sjoin2_final["Tamanio"] = gdf_sjoin2_final["Tamanio"].astype(int)

In [45]:
gdf_sjoin2_final.sample(2)

,ID_Distrito,Precio,Tipo,Planta,Tamanio,Habitaciones,Banios,Direccion,Descripcion,geometry
281,3,1600.0,departamento,bj,56.0,1,1,Avenida De Menéndez Pelayo,Urbanissimo Real Estate Comercializa Apartamen...,POINT (-3.67598 40.41235)
372,15,1700.0,departamento,bj,90.0,3,1,Calle Del Buen Gobernador,Se Alquila Luminoso Piso En El Céntrico Barrio...,POINT (-3.65578 40.4334)


In [164]:
gdf_sjoin2_final.dtypes

ID_Distrito        int32
Precio           float64
Tipo              object
Planta            object
Tamanio            int64
Habitaciones       int64
Banios             int64
Direccion         object
Descripcion       object
geometry        geometry
dtype: object

In [ ]:
#gdf_sjoin2_final.to_file('../datos/finales/idealista.geojson', driver='GeoJSON')

## WebScraping Ayuntamiento de Madrid

- Usaremos el Banco de Datos del Ayuntamiento de Madrid para obtener información de población nacional y extranjera los municipios de Madrid. En específico, nos interesa la serie 6.1.2.B, que muestra la evolución anual de la población por distrito según Nacionalidad. Se puede consultar la web en [este enlace](https://servpub.madrid.es/CSEBD_WBINTER/seleccionSerie.html?numSerie=0307010000022).

- El objetivo será descargar un archivo de tipo CSV seleccionando las opciones necesarias. Lo haremos haciendo uso de la función `sf.scraping_ayuntamiento()`, que no tiene parámetros de entrada, y descarga directamente a un directorio del ordenador el archivo.

- Si deseas probarla, debes entrar al `../src/soporte_funciones` y cambiar la ruta de descarga (ruta relativa) para que coincida con la carpeta donde has copiado este repositorio.

In [ ]:
#sf.scraping_ayuntamiento()

Cookies aceptadas
Click en todos los distritos
Click en totales barrios
Click en todos los períodos
Click en todas las medidas
Click en todas las nacionalidades
Click en generar CSV


- Verificamos la descarga y haciendo uso de la libería `os` renombraremos el archivo.

In [59]:
os.listdir("../datos/origen/")

['airbnb.json',
 '.DS_Store',
 '31097.csv',
 'Datos de la serie 0307010000022.csv',
 'madrid-districts.geojson',
 'sopas_redpiso.pkl',
 'airbnb.geojson',
 'idealista.geojson',
 'idealista.json']

In [ ]:
#os.rename("../datos/origen/Datos de la serie 0307010000022.csv", "../datos/origen/extranjeros_madrid.csv")

- Ahora debemos importarlo a nuestro notebook y convertirlo en un dataframe.

In [ ]:
df_extranjeros = pd.read_csv("../datos/origen/extranjeros_madrid.csv", sep=",", encoding="latin-1")

- Sobre este, realizaremos tareas de transformación.

In [4]:
# Extraemos la lista de nombres de las columnas para modificarlos
encabezados_extranjeros = df_extranjeros.columns
# Nos quedamos sólamente con los códigos de cada distrito
nuevos_encabezados_ext = [re.findall(r"\d+", enc)[0] if re.findall(r"\d+", enc) else enc for enc in encabezados_extranjeros]
# Eliminamos los ceros delante de cada distrito.
nuevos_encabezados_ext2 = [enc.lstrip("0") for enc in nuevos_encabezados_ext]
# Reasignamos los nombres de las columnas.
df_extranjeros.columns = nuevos_encabezados_ext2
# Solucionamos problemas relacionados con el encoding.
df_extranjeros["Categoria"] = df_extranjeros["Categoria"].str.replace("Espaola","Espaniola")
df_extranjeros.sample(3)

,Periodo,Categoria,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
14,2022,Extranjera,519,-338,96,622,-247,-713,146,-645,-351,-966,-1219,-1170,-1209,-121,-646,332,-405,298,527,-77,-84
6,2020,Total,5677,1912,1186,1827,1808,3497,1483,3996,2375,4022,7321,3456,6248,1089,3737,5139,5638,4189,1941,3070,1088
10,2021,Espaniola,-2015,-2147,-2364,-2916,-2415,-2241,-3048,-2783,-1441,-3706,-3386,-1600,-3875,-1975,-3879,-965,-911,-692,970,-2024,-152


- Dado que cargaremos estos datos a una base de datos de SQL, nos interesa que el formato de los datos sea adecuado para trabajar con claves primarias y foráneas. En este caso, los IDs de los municipios serán claves foráneas, por lo que deben estar en las filas.
- Para conseguir eso, convertiremos el DF a un formato largo (melted) y cambiaremos su forma (reshape).

In [5]:
#Convertir a formato largo
df_ext_melted = pd.melt(df_extranjeros, id_vars=["Periodo", "Categoria"], var_name="ID_Distrito", value_name="Habitantes")
df_ext_melted

,Periodo,Categoria,ID_Distrito,Habitantes
0,2018,Total,1,359
1,2018,Espaniola,1,-631
2,2018,Extranjera,1,991
3,2019,Total,1,2533
4,2019,Espaniola,1,806
...,...,...,...,...
310,2021,Espaniola,21,-152
311,2021,Extranjera,21,78
312,2022,Total,21,129
313,2022,Espaniola,21,214


In [6]:
# Pivotar para obtener una fila por distrito y por año
df_ext_reshaped = df_ext_melted.pivot_table(index=["ID_Distrito", "Periodo"], columns="Categoria", values="Habitantes").reset_index()
# Cambiar tipos de datis
df_ext_reshaped= df_ext_reshaped.astype(int)
df_ext_reshaped

Categoria,ID_Distrito,Periodo,Espaniola,Extranjera,Total
0,1,2018,-631,991,359
1,1,2019,806,1728,2533
2,1,2020,2362,3314,5677
3,1,2021,-2015,2637,622
4,1,2022,-1947,519,-1430
...,...,...,...,...,...
100,9,2018,740,177,913
101,9,2019,704,859,1566
102,9,2020,1187,1189,2375
103,9,2021,-1441,281,-1161


- Con este nuevo formato, comprobamos los tipos de datos y exportamos a un archivo CSV.

In [7]:
df_ext_reshaped.dtypes

Categoria
ID_Distrito    int64
Periodo        int64
Espaniola      int64
Extranjera     int64
Total          int64
dtype: object

In [ ]:
#df_ext_reshaped.to_csv("../datos/finales/poblacion.csv")

## WebScraping INE

- Haremos uso de la serie "Indicadores de renta media y mediana", parte de la encuesta de Condiciones de Vida del Instituto Nacional de Estadística, para obtener sobre la renta media por hogar en los municipios de Madrid. Se puede consultar la web en [este enlace](https://www.ine.es/jaxiT3/Tabla.htm?t=31097).

- El objetivo será descargar un archivo de tipo CSV seleccionando las opciones necesarias. Lo haremos haciendo uso de la función `sf.scraping_ine()`, que no tiene parámetros de entrada, y descarga directamente a un directorio del ordenador el archivo.

- Si deseas probarla, debes entrar al `../src/soporte_funciones` y cambiar la ruta de descarga (ruta relativa) para que coincida con la carpeta donde has copiado este repositorio.

In [ ]:
#sf.scraping_ine()

Cookies aceptadas
Quitadas opciones por defecto
Click en todos los años
Desplegable Madrid abierto
Click en distritos
Click en descarga
Click en CSV


- Verificamos la descarga y haciendo uso de la libería `os` renombraremos el archivo.

In [61]:
os.listdir("../datos/origen/")

['airbnb.json',
 '.DS_Store',
 '31097.csv',
 'extranjeros_madrid.csv',
 'madrid-districts.geojson',
 'sopas_redpiso.pkl',
 'airbnb.geojson',
 'idealista.geojson',
 'idealista.json']

In [ ]:
#os.rename("../datos/origen/31097.csv", "../datos/origen/ingresos_hogares_distrito.csv")

- Importamos el archivo y realizamos tareas de transformación, para posteriormente verificar los tipos de datos y exportar el resultado final a un archivo CSV.

In [31]:
# Lectura
df_renta = pd.read_csv("../datos/origen/ingresos_hogares_distrito.csv", sep=";")
# Eliminar columnas
df_renta.drop(columns=["Municipios", "Secciones", "Indicadores de renta media y mediana"], inplace=True)
# Corregir ubicación del punto
df_renta["Total"] = df_renta["Total"]*1000
# Quedarnos sólamente con los números de distrito
df_renta['Distritos'] = df_renta['Distritos'].str.extract(r'(\d{2})$', expand=False)
# Eliminar los ceros delante de los números.
df_renta["Distritos"] = df_renta["Distritos"].apply(lambda x: x.lstrip("0") if isinstance(x, str) else x)
# Renombrar columnas
df_renta.rename(columns={"Distritos":"ID_Distrito"}, inplace=True)
# Cambiar tipos de datos
df_renta["ID_Distrito"] = df_renta["ID_Distrito"].astype(int)
df_renta

,ID_Distrito,Periodo,Total
0,1,2022,41059.0
1,1,2021,38360.0
2,1,2020,36984.0
3,1,2019,38171.0
4,1,2018,36072.0
...,...,...,...
163,21,2019,51108.0
164,21,2018,49615.0
165,21,2017,47782.0
166,21,2016,47389.0


In [32]:
df_renta.dtypes

ID_Distrito      int64
Periodo          int64
Total          float64
dtype: object

In [ ]:
#df_renta.to_csv("../datos/finales/ingresos_hogares.csv")

- Con esta tarea finalizamos la extracción y transformación de los datos. Continuaremos en el notebook #2 con la creación e inserción de los datos a una base de datos SQL.